<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [1]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import glob
import datetime
import matplotlib

In [2]:
!pip install geopandas
import geopandas as gpd

# Data Preparation

## Download MBTA Data

In [3]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1"):
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv"):
  !unzip data.1

## Load MBTA Arrival Data

In [4]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


## Utility functions

In [5]:
def filter_on_row_col(df, target_rows, target_columns):
  for key, value in target_rows:
      df.drop(df[df[key] != value].index, inplace=True)
  df.drop(df.columns.difference(target_columns), axis=1, inplace=True)

def load_targets_from_csvs(target_columns, target_rows):
  bus_arrival_files = glob.glob('./*.csv')

  df_cumulative = pd.DataFrame()
  for f in bus_arrival_files:
    df_tmp = pd.read_csv(f)
    filter_on_row_col(df_tmp, target_rows, target_columns)
    df_cumulative = df_cumulative.append(df_tmp)
    print(f"Loaded {f}")
    del df_tmp

  return df_cumulative

# EDA on Routes Timeliness

In [6]:
df2 = load_targets_from_csvs(target_columns=['route_id','direction','stop_id'], target_rows=[])

Loaded .\MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded .\MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded .\MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded .\MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2019.csv


In [7]:
# Calculate visits per stop
group = df2[['route_id','direction','stop_id']].groupby(['route_id','direction'])
unique_stops = group.nunique().rename(columns={"stop_id":"unique_stop_ids"})
total_stops = pd.DataFrame(group.size()).rename(columns={0:"total_stops"})
stops = pd.merge(unique_stops, total_stops, left_index=True, right_index=True)
stops['visits_per_stop'] = stops['total_stops'] / stops['unique_stop_ids']

In [8]:
stops.sort_values('visits_per_stop', ascending=False).head(20)

unique_stop_ids  total_stops  visits_per_stop
route_id direction                                               
SL5      Outbound                12       814194     67849.500000
         Inbound                 13       872944     67149.538462
111      Inbound                 12       777438     64786.500000
SL1      Outbound                 7       448292     64041.714286
31       Outbound                 5       304750     60950.000000
         Inbound                  5       300727     60145.400000
SL2      Outbound                 6       345890     57648.333333
         Inbound                  6       335191     55865.166667
111      Outbound                13       718718     55286.000000
23       Inbound                 10       549635     54963.500000
SL1      Inbound                 13       709980     54613.846154
01       Outbound                 9       488393     54265.888889
         Inbound                  9       472691     52521.222222
39       Inbound                 11       561718     51065.272727
28       Inbound                 11       552235     50203.181818
39       Outbound                10       496903     49690.300000
743      Outbound                11       544599     49509.000000
77       Outbound                11       538967     48997.000000
         Inbound                 11       536599     48781.727273
23       Outbound                12       548555     45712.916667

# Preprocess data

Loading the data for a single route and a single direction

In [9]:
target_columns = ["service_date", "half_trip_id", "stop_id", "stop_sequence", "scheduled", "actual"]
target_rows = [('route_id', '01'),('direction', 'Inbound')]
df3 = load_targets_from_csvs(target_columns, target_rows)

Loaded .\MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded .\MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded .\MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded .\MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2019.csv


Comment:
stop_id and stop_sequence are not always consistent in the dataframe!

In [10]:
group = df3.groupby(by=['stop_id', 'stop_sequence']).size()
print(group)

stop_id  stop_sequence
59       5.0                  1
         6.0                  3
         7.0              42083
62       1.0                  2
         6.0                  1
         7.0                  3
         8.0              42080
64       1.0                180
         2.0                  2
         7.0                  1
         8.0                  3
         9.0              42079
67       1.0                  3
         2.0              42069
72       1.0                  1
         2.0                  3
         3.0              42069
75       2.0                  1
         3.0                  3
         4.0              42083
79       3.0                  1
         4.0                  3
         5.0              42083
110      1.0              42070
187      4.0                  1
         5.0                  3
         6.0              42083
dtype: int64


Create a definition of valid stop IDs, based on the frequency of occurence

In [11]:
valid_stop_ids = group[group > group.max()/2]
sequence_to_id = valid_stop_ids.index.to_frame().reset_index(drop=True).sort_values(['stop_sequence']).set_index(['stop_id'])
print(sequence_to_id)
sequence_to_id = sequence_to_id['stop_sequence'].to_dict()

         stop_sequence
stop_id               
110                1.0
67                 2.0
72                 3.0
75                 4.0
79                 5.0
187                6.0
59                 7.0
62                 8.0
64                 9.0


Drop invalid (out of sequence) stop IDs

In [12]:
df3['expected_stop_sequence'] = df3['stop_id'].apply(lambda x: sequence_to_id[x])
df3.drop(df3[((df3['expected_stop_sequence'] != df3['stop_sequence']) | (df3['stop_sequence'].isna()))].index, axis=0, inplace=True)
df3.drop('expected_stop_sequence', axis=1, inplace=True)

Pivot data frame

In [13]:
df4 = df3.pivot(index=['service_date', 'half_trip_id'], columns=['stop_sequence'], values=['scheduled',	'actual'])

Drop incomplete routes

In [14]:
df5 = df4[df4.isna().sum(axis=1) == 0]

## Transform data

In [15]:
# Transform to datetime
time_format = '%Y-%m-%d %H:%M:%S'
df5 = df5.applymap(lambda x: datetime.datetime.strptime(x, time_format))

# Transform to seconds from day start
since = datetime.datetime(1900, 1, 1, 0, 0)
df6 = df5.applymap(lambda x: (x - since).total_seconds() )
df6

scheduled                                      \
stop_sequence                   1.0      2.0      3.0      4.0      5.0   
service_date half_trip_id                                                 
2018-08-02   40135101.0     18600.0  18780.0  18960.0  19140.0  19320.0   
             40135103.0     22860.0  23040.0  23220.0  23400.0  23700.0   
             40135105.0     27840.0  28140.0  28500.0  28800.0  29100.0   
             40135121.0     25560.0  25800.0  25980.0  26220.0  26520.0   
             40135155.0     21960.0  22140.0  22320.0  22500.0  22680.0   
...                             ...      ...      ...      ...      ...   
2019-09-30   45106583.0     59040.0  59340.0  59700.0  60120.0  60660.0   
             45106585.0     66780.0  67080.0  67440.0  67740.0  68160.0   
             45106587.0     72660.0  72900.0  73140.0  73440.0  73740.0   
             45106589.0     77580.0  77820.0  78060.0  78360.0  78660.0   
             45108654.0     65880.0  66180.0  66540.0  66960.0  67500.0   

                                                                actual  \
stop_sequence                  6.0      7.0      8.0      9.0      1.0   
service_date half_trip_id                                                
2018-08-02   40135101.0    19500.0  19680.0  19920.0  19980.0  18705.0   
             40135103.0    23940.0  24120.0  24540.0  24600.0  22880.0   
             40135105.0    29460.0  29760.0  30240.0  30300.0  27811.0   
             40135121.0    26760.0  26940.0  27420.0  27480.0  25558.0   
             40135155.0    22860.0  23040.0  23280.0  23340.0  21992.0   
...                            ...      ...      ...      ...      ...   
2019-09-30   45106583.0    61140.0  61500.0  61980.0  62400.0  59046.0   
             45106585.0    68520.0  68820.0  69120.0  69480.0  66555.0   
             45106587.0    74040.0  74280.0  74520.0  74760.0  72622.0   
             45106589.0    78960.0  79200.0  79440.0  79680.0  77501.0   
             45108654.0    67860.0  68160.0  68460.0  68820.0  65909.0   

                                                                        \
stop_sequence                  2.0      3.0      4.0      5.0      6.0   
service_date half_trip_id                                                
2018-08-02   40135101.0    18931.0  19120.0  19397.0  19638.0  19766.0   
             40135103.0    23195.0  23384.0  23624.0  23854.0  24112.0   
             40135105.0    28245.0  28727.0  29157.0  29775.0  30091.0   
             40135121.0    25893.0  26169.0  26500.0  26799.0  27073.0   
             40135155.0    22219.0  22438.0  22684.0  23052.0  23414.0   
...                            ...      ...      ...      ...      ...   
2019-09-30   45106583.0    59421.0  59931.0  60355.0  60798.0  61164.0   
             45106585.0    66919.0  67259.0  67521.0  67926.0  68220.0   
             45106587.0    72949.0  73223.0  73519.0  73812.0  74210.0   
             45106589.0    77805.0  78010.0  78306.0  78543.0  78706.0   
             45108654.0    66392.0  66658.0  67056.0  67322.0  67600.0   

                                                      
stop_sequence                  7.0      8.0      9.0  
service_date half_trip_id                             
2018-08-02   40135101.0    20032.0  20237.0  20260.0  
             40135103.0    24290.0  24607.0  24637.0  
             40135105.0    30316.0  30965.0  31031.0  
             40135121.0    27298.0  27733.0  27781.0  
             40135155.0    23550.0  24049.0  24092.0  
...                            ...      ...      ...  
2019-09-30   45106583.0    61596.0  62147.0  62196.0  
             45106585.0    68553.0  68912.0  68983.0  
             45106587.0    74459.0  74723.0  74759.0  
             45106589.0    79002.0  79203.0  79224.0  
             45108654.0    67955.0  68440.0  68503.0  

[37203 rows x 18 columns]

In [16]:
# Create 'timeliness' metric (seconds early or late)
df7 = df6[('actual',)].loc[:] - df6[('scheduled',)].loc[:]
columns = [('timeliness', col) for col in df7.columns]
df7.columns = pd.MultiIndex.from_tuples(columns, names=[None, 'stop_sequence'])

In [17]:
# Create timeliness dataframe
df8 = pd.concat([df5, df7], axis=1, sort=False)
df8.drop('actual', axis=1, level=0, inplace=True)
df8['route_start_time'] = df8[("scheduled", 1.0)]
df8 = df8.reset_index().set_index(["service_date", "route_start_time"])
df8.drop('scheduled', axis=1, level=0, inplace=True)
df8.drop('half_trip_id', axis=1, level=0, inplace=True)
df8

timeliness                              \
stop_sequence                           1.0    2.0    3.0    4.0    5.0   
service_date route_start_time                                             
2018-08-02   1900-01-01 05:10:00      105.0  151.0  160.0  257.0  318.0   
             1900-01-01 06:21:00       20.0  155.0  164.0  224.0  154.0   
             1900-01-01 07:44:00      -29.0  105.0  227.0  357.0  675.0   
             1900-01-01 07:06:00       -2.0   93.0  189.0  280.0  279.0   
             1900-01-01 06:06:00       32.0   79.0  118.0  184.0  372.0   
...                                     ...    ...    ...    ...    ...   
2019-09-30   1900-01-01 16:24:00        6.0   81.0  231.0  235.0  138.0   
             1900-01-01 18:33:00     -225.0 -161.0 -181.0 -219.0 -234.0   
             1900-01-01 20:11:00      -38.0   49.0   83.0   79.0   72.0   
             1900-01-01 21:33:00      -79.0  -15.0  -50.0  -54.0 -117.0   
             1900-01-01 18:18:00       29.0  212.0  118.0   96.0 -178.0   

                                                              
stop_sequence                       6.0    7.0    8.0    9.0  
service_date route_start_time                                 
2018-08-02   1900-01-01 05:10:00  266.0  352.0  317.0  280.0  
             1900-01-01 06:21:00  172.0  170.0   67.0   37.0  
             1900-01-01 07:44:00  631.0  556.0  725.0  731.0  
             1900-01-01 07:06:00  313.0  358.0  313.0  301.0  
             1900-01-01 06:06:00  554.0  510.0  769.0  752.0  
...                                 ...    ...    ...    ...  
2019-09-30   1900-01-01 16:24:00   24.0   96.0  167.0 -204.0  
             1900-01-01 18:33:00 -300.0 -267.0 -208.0 -497.0  
             1900-01-01 20:11:00  170.0  179.0  203.0   -1.0  
             1900-01-01 21:33:00 -254.0 -198.0 -237.0 -456.0  
             1900-01-01 18:18:00 -260.0 -205.0  -20.0 -317.0  

[37203 rows x 9 columns]

# Training

## TODO

# Inference

## TODO